In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(100)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.make.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Year'] = df_input['Year'].astype(int)
df_input['Make No'] = df_input['Make No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        data = {'action': 'bf_ajax_hook',
                'type': 'model',
                'c_year': crawler_series['Year'],
                'c_make': crawler_series['Make']}

        count_retry = 0
        while True:
            count_retry += 1

            try:
                url_check = 'https://www.rainx.com/'

                resp = requests.post('https://www.rainx.com/wp-admin/admin-ajax.php',
                                     data=data,
                                     headers=get_header(),
                                     proxies=get_proxy(),
                                     timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    if html.xpath('//option/text()')[0].strip() == 'Model':
                        break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        list_model = [text for text in html.xpath('//option/text()')[1:]]

        # = = = = = = = = = = = = = = =

        with semaphore_correct:
            list_dict_correct.extend([{'Year': crawler_series['Year'],
                                       'Make No': crawler_series['Make No'],
                                       'Make': crawler_series['Make'],
                                       'Model No': i + 1,
                                       'Model': model} for i, model in enumerate(list_model)])

        # = = = = = = = = = = = = = = =

        crawler_status = True
    except:
        crawler_series['Check_Url'] = url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['Year']} -> {crawler_series['Make No']}. {crawler_series['Make']}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Year', 'Make No', 'Model No'],
                                                             ascending=[False, True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/3.model-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Year', 'Make No'],
                                                           ascending=[False, True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/model_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[True] - [请求1次] - [剩余299条] - [2024/11/20 09:08:41] - 1976 -> 4. BMW

[True] - [请求2次] - [剩余298条] - [2024/11/20 09:08:41] - 1979 -> 31. Toyota

[True] - [请求1次] - [剩余297条] - [2024/11/20 09:08:41] - 1976 -> 8. Chrysler

[True] - [请求2次] - [剩余296条] - [2024/11/20 09:08:41] - 1979 -> 32. Volkswagen

[True] - [请求1次] - [剩余295条] - [2024/11/20 09:08:42] - 1976 -> 7. Chevrolet

[True] - [请求2次] - [剩余294条] - [2024/11/20 09:08:42] - 1978 -> 18. Mazda

[True] - [请求1次] - [剩余293条] - [2024/11/20 09:08:42] - 1976 -> 11. Ford

[True] - [请求3次] - [剩余292条] - [2024/11/20 09:08:42] - 1981 -> 32. Toyota

[True] - [请求1次] - [剩余291条] - [2024/11/20 09:08:42] - 1976 -> 12. GMC

[True] - [请求1次] - [剩余290条] - [2024/11/20 09:08:42] - 1976 -> 13. Honda

[True] - [请求3次] - [剩余289条] - [2024/11/20 09:08:43] - 1980 -> 31. Volkswagen

[True] - [请求1次] - [剩余288条] - [2024/11/20 09:08:43] - 1976 -> 10. Ferrari

[True] - [请求1次] - [剩余287条] - [2024/11/20 09:08:43] - 1976 -> 9. Dodge

[True] - [请求1次] - [剩余286条] - [2024/11/20 09:08:43] -

C:\Users\Administrator\Desktop\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
